In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/CS769

/content/drive/.shortcut-targets-by-id/1pyPDXXgFYu_gD-IDMCmfmchXQboVGhNs/CS769


In [ ]:
!pip install emoji transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 81.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 85.4 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=b19ee79af61ed4c2a3246923130522a172d438dfdcf5ccc8b03767b1ec1c7e24
  Stored in directory: /root/.cache/pip/wheels/02/3d/88/51a592b9ad17e7899126563698b4e3961983ebe85747228ba6
Successfully built emoji


In [ ]:
import pandas as pd
import emoji
import string

In [ ]:
df_ck_train =  pd.read_csv("original/ck_train.csv", encoding='utf-8')
df_ck_test =  pd.read_csv("original/ck_test.csv", encoding='utf-8')
df_ck_dev =  pd.read_csv("original/ck_dev.csv", encoding='utf-8')
df_k_train = pd.read_csv("translated/k_train.csv", encoding='utf-8')
df_e_train = pd.read_csv("translated/e_train.csv", encoding='utf-8')

In [ ]:
concatenated = df_ck_train['text'].str.cat(sep=' ')
characters_no_emoji = {c for c in set(concatenated) if not emoji.is_emoji(c)}
vocabulary = list(characters_no_emoji)
print(len(vocabulary))

225


In [ ]:
clean_vocabulary = set(vocabulary) - set(string.punctuation) - set(string.whitespace) - {'\xa0','\u200b','\u200c','\u200d','’','“','”','₹','℅'}

In [ ]:
vocabulary=list(clean_vocabulary)

In [ ]:
## Method to swap characters
import random
import string
import emoji

import random
import string
import emoji

def augment_sentence_swapping(sentence, p=0.5):
    # Split sentence into words and process each word separately
    words = sentence.split()
    for i, word in enumerate(words):
        # Randomly decide whether to swap one adjacent character
        if random.random() < p:
            # Check if the word has at least two non-space/non-emoji characters
            chars = [c for c in word if c not in string.whitespace and not emoji.is_emoji(c)]
            if len(chars) >= 2:
                # Choose a random non-space/non-emoji character position to swap
                pos = random.randint(0, len(chars)-2)
                # Swap the chosen character with the next character
                new_word = ''
                j = 0
                while j < len(word):
                    if word[j] not in string.whitespace and not emoji.is_emoji(word[j]):
                        if j == pos:
                            new_word += word[j+1] + word[j]
                            j += 2
                        elif j == pos + 1:
                            j += 1
                        else:
                            new_word += word[j]
                            j += 1
                    else:
                        new_word += word[j]
                        j += 1
                words[i] = new_word
    # Join the processed words back into a sentence
    augmented_sentence = ' '.join(words)
    return augmented_sentence

def augment_sentence_substitute(sentence, char_list, p=0.5):
    # Split sentence into words and process each word separately
    words = sentence.split()
    for i, word in enumerate(words):
        # Randomly decide whether to substitute a character
        if random.random() < p:
            # Check if the word has at least one non-space/non-emoji character
            chars = [c for c in word if c not in string.whitespace and not emoji.is_emoji(c)]
            if len(chars) >= 1:
                # Choose a random non-space/non-emoji character position to substitute
                pos = random.randint(0, len(chars)-1)
                # Choose a random character from the given list to substitute with
                new_char = random.choice(char_list)
                # Substitute the chosen character with the new character
                new_word = ''
                j = 0
                while j < len(word):
                    if word[j] not in string.whitespace and not emoji.is_emoji(word[j]):
                        if j == pos:
                            new_word += new_char
                            j += 1
                        else:
                            new_word += word[j]
                            j += 1
                    else:
                        new_word += word[j]
                        j += 1
                words[i] = new_word
    # Join the processed words back into a sentence
    augmented_sentence = ' '.join(words)
    return augmented_sentence


def augment_sentence_deleteChar(sentence, p=0.5):
    # Split sentence into words and process each word separately
    words = sentence.split()
    for i, word in enumerate(words):
        # Randomly decide whether to delete a character
        if random.random() < p:
            # Check if the word has at least two characters
            if len(word) >= 2:
                # Choose a random character position to delete
                pos = random.randint(0, len(word)-1)
                # Delete the chosen character
                new_word = word[:pos] + word[pos+1:]
                words[i] = new_word
    # Join the processed words back into a sentence
    augmented_sentence = ' '.join(words)
    return augmented_sentence


def augment_sentence_insert(sentence, char_list, p=0.5):
    # Split sentence into words and process each word separately
    words = sentence.split()
    for i, word in enumerate(words):
        # Randomly decide whether to insert a character
        if random.random() < p:
            # Choose a random character from the given list
            char = random.choice(char_list)
            # Choose a random position to insert the character
            pos = random.randint(0, len(word))
            # Insert the chosen character at the chosen position
            new_word = word[:pos] + char + word[pos:]
            words[i] = new_word
    # Join the processed words back into a sentence
    augmented_sentence = ' '.join(words)
    return augmented_sentence

def augment_sentence(sentence, char_list, p=0.5):
    # Define the four operations as lambda functions
    insert_op = lambda word: augment_sentence_insert(word, char_list, p)
    delete_op = lambda word: augment_sentence_deleteChar(word)
    substitute_op = lambda word: augment_sentence_substitute(word, char_list, p)
    swap_op = lambda word: augment_sentence_swapping(word)
    # Define a list of the four operations
    ops = [insert_op, delete_op, substitute_op, swap_op]
    # Split sentence into words and process each word separately
    words = sentence.split()
    for i, word in enumerate(words):
        # Choose a random operation and apply it to the word
        random_op = random.choice(ops)
        words[i] = random_op(word)
    # Join the processed words back into a sentence
    augmented_sentence = ' '.join(words)
    return augmented_sentence

def generate_augmented_sentences(row):
    sentences = [row['text']]
    labels = [row['label']]
    for i in range(3):
        sentences.append(augment_sentence(row['text'], vocabulary))
        labels.append(row['label'])
    return pd.concat([pd.Series(sentences), pd.Series(labels)], axis=1)

In [ ]:
augment_sentence_insert("Hello", vocabulary)

'Heಢllo'

In [ ]:
augmented_data = []
for i, row in df_ck_train.iterrows():
  augmented_data.append({'text': row['text'], 'label': row['label']})
  for _ in range(3):
    augmented_data.append({'text': augment_sentence(row['text'], vocabulary), 'label': row['label']})
augmented_df = pd.DataFrame(augmented_data)

In [ ]:
rm augmented/cka_train.csv

In [ ]:
augmented_df.to_csv('augmented/cka_train.csv',index=False)